# Laboratoire 3 : Machines à vecteurs de support et réseaux neuronaux
#### Département du génie logiciel et des technologies de l’information

| Étudiants             |                                                         |
|-----------------------|---------------------------------------------------------|
| Jean-Philippe Decoste |  DECJ19059105                                           |
| Ahmad Al-Taher        |   ALTA22109307                                          |
| Stéphanie Lacerte     |   LACS06629109                                          |
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2018                                            |
| Groupe                | 2                                                       |
| Numéro du laboratoire | 02                                                      |
| Professeur            | Hervé Lombaert                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 30 oct 2018                                             |

In [ ]:
import csv
import math
import os

import graphviz
import matplotlib.pyplot as plt
import numpy as np
from imageV2 import Image as imageFeat
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.svm import SVC

from spam import Spam
import utilities
from tabulate import tabulate

In [13]:
def prepareDataset():
    print("PREPARING DATASETS")
    allData_length = len(list(csv.reader(open(r"galaxies.csv"))))
    progress = 0
    datas = []

    print("Reading Galaxies features:")
    utilities.printProgressBar(0, allData_length, prefix='Progress:', suffix='Complete', length=50)
    with open(r"galaxies.csv", 'r') as theFile:
        primitives = csv.reader(theFile, delimiter=',', quotechar='|')

        for row in primitives:
            progress += 1
            utilities.printProgressBar(progress+1, allData_length, prefix='Progress', suffix='Complete', length=50)

            values = [float(i) for i in row]
            datas.append(imageFeat(values))
    print("\n-> Done\n")

    #3. Split dataset using model_selection
    for data in np.array(datas):
        features.append(data.features)
        answers.append(data.answer)

## SVM

In [23]:
def svm():
    
    validation = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
    #linear
    params = dict(kernel=['linear'],c=[0.001,0.1,1.0,10.0], class_weight=['balanced'], cache_zide=[2048])
    grid = GridSearchCV(SVC(), param_grid=params, cv=validation, n_jobs=-1, iid=True)
    #Fit the feature to svm algo
    grid.fit(features, answers)
    
    print( grid.best_score_)
    
    #print raw 
    print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
    #rbf
    
    #print(tabulate(bayesPerf, headers="firstrow"))
    
    print("-> Done\n\n")

## Réseaux neuronaux

## Main

In [ ]:
#1.A Read Galaxy features (name of file, path, n_split, test size, random state)
features = []
answers = []

prepareDataset()

#svm()


PREPARING DATASETS
Reading Galaxies features:
Progress |**********----------------------------------------| 21.0% Complete

## Introduction

## Question 1 
### Présentation de la méthode de validation

## Question 2
### Description du modèle élaboré

## Question 3
### Analyse de la phase d'entraînement

## Question 4
### Matrice des expérimentations

### Étude des hyperparamètres des modèles

## Question 5
### Matrice des expérimentations

### Études des hyperparamètres des modèles

## Question 6
### Discussion de l'impact de la taille des données

## Question 7
### Discussion de l'impact du bruit

## Question 8
### Formulation des recommandations

## Question 9
### Améliorations possibles

## Conclusion

## Bibliographie